In [2]:
import numpy as np
import pandas as pd
music_log1, music_log2, music_log3 = ['20170301_play.log', '20170302_1_play.log', '20170303_1_play.log']
#df = pd.read_table(music_log1, sep='\t', skiprows=2, nrows=500000, header=None)
# df = pd.read_csv(music_log1, sep='\t', skiprows=2, nrows=5000, header= None,names=['uid', 'os', 'rid', 'type', 'name', 'artist', 
# 'play_len', 'song_len', 'TBA'],  error_bad_lines=True, warn_bad_lines=True)
df = pd.read_csv(music_log1, sep='\t', header= None, skiprows=2, error_bad_lines=False, warn_bad_lines=False)

In [3]:
df.columns=['uid', 'os', 'rid', 'type', 'name', 'artist', 'play_len', 'song_len', 'pay']

In [4]:
df.head()

,uid,os,rid,type,name,artist,play_len,song_len,pay
0,154422682.0,ar,20870993,1,用情,狮子合唱团,22013,332,0.0
1,154421907.0,ip,6560858,0,表情不要悲伤,伯贤&D.O.&张艺兴&朴灿烈,96,161,0.0
2,154422630.0,ar,3385963,1,"Baby, Don't Cry(人鱼的眼泪)",EXO,235868,235,0.0
3,154410267.0,ar,6777172,0,3D-环绕音律1(3D Mix),McTaiM,164,237,0.0
4,154407793.0,ar,19472465,0,刚好遇见你,曲肖冰,24,201,0.0


In [5]:
df.shape

(3412897, 9)

In [6]:
df.drop('type', axis=1, inplace=True)
df.drop('artist', axis=1, inplace=True)
df.drop('pay', axis=1, inplace=True)

In [7]:
df.describe()

//anaconda/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,uid
count,3.412220e+06
mean,1.471100e+08
std,3.085694e+07
min,0.000000e+00
25%,NaN
50%,NaN
75%,NaN
max,1.548342e+08


In [8]:
df.isnull().sum()

uid          677
os             4
rid            4
name           4
play_len    1358
song_len    1358
dtype: int64

In [9]:
# #Check if there is "NULL" UID rows
# import numpy as np

# row_num = df.shape[0]
#null_list=[]
# for i in range(row_num):
#     if(np.array(df['uid'].isnull())[i]==True):
#         print("UID is NA in row #{} ".format(i))
#         print df.ix[i]
#         print("\n")
#         null_list.append(i)
#         #df.drop(i, axis=0, inplace=True)
#     elif(np.array(df['rid'].isnull())[i] ==True):
#         print("RID is NA in row #{} ".format(i))
#         print df.ix[i]
#         print("\n")
#         null_list.append(i)
#     elif(np.array(df['play_len'].isnull())[i] ==True):
#         print("Play_len is NA in row #{} ".format(i))
#         print df.ix[i]
#         print("\n")
#         null_list.append(i)
#     elif(np.array(df['song_len'].isnull())[i] ==True):
#         print("Song_len is NA in row #{} ".format(i))
#         print df.ix[i]
#         print("\n")
#         null_list.append(i)
#        df.drop(i, axis=0, inplace=True)
# for j in range(len(null_list)):
#         df.drop(null_list[j], axis=0, inplace=True)
#df.dropna(axis=0, inplace=True)
#na_free = df.dropna()
# df_na = df[~df.index.isin(na_free.index)]
# null_list = list(df_na.index)
# df=na_free
df.dropna(inplace=True)
df['play_len']=pd.to_numeric(df['play_len'], errors=coerce)
df['song_len']=pd.to_numeric(df['song_len'], errors=coerce)
df['uid']=pd.to_numeric(df['uid'], errors=coerce)
df['rid']=pd.to_numeric(df['rid'], errors=coerce)
# df=df.fillna(0)

In [10]:
df.shape

(3411539, 6)

In [13]:
import numpy as np
df.song_len.replace('0',np.nan, inplace = True)

In [15]:
#impute missing song_len with existing value
df['song_len'] = df.groupby(['rid','name'])['song_len']\
.transform(lambda grp: grp.fillna(method='ffill'))

In [16]:
delete_index = df[df['song_len'].isnull()].index
df = df.drop(delete_index, axis = 0)

In [17]:
df.shape

(3323808, 6)

In [ ]:
# df[df.rid==0].rid = df[.index]
# l=np.zeros(df[df.rid==0].shape[0])
for index, i in df[df.rid==0].iterrows():
    df.drop(index,axis=0, inplace=True)
    

In [35]:
len(df[df.rid==0].index)

35691

In [25]:
#Easy version: delete 0 rows that cannot be imputed
import numpy as np

# uid_zero_list=[]
# rid_zero_list=[]
# song_len_zero_list=[]
# new_rid=1
# new_uid=1
for index, i in df.iterrows():
        if(df.ix[i,'uid']==0): #re-assign uid=0 rows
# #            impute_flag0=0
#             while(not (new_rid in list(df['uid']))):
#                 new_uid+=1
#             df.ix[i,'uid']=new_uid
#             new_uid+=1
#             print("UID in row #{} is re-assigned as {}".format(i, new_uid))
#             impute_flag0=1
#             if(impute_flag0==0):
#             print("No way to impute UID in row #{} ".format(i))
#             print df.ix[i]
#             print("\n")
            df.drop(i, inplace=True)
#             uid_zero_list.append(i)
        elif(df.ix[i,'rid']==0): #impute rid=0 rows
            impute_flag=0
            for index, j in df.iterrows():
                if(df.ix[j,'name']==df.ix[i,'name'] and j!=i and df.ix[j,'artist'] == df.ix[i,'artist'] and df.ix[j,'rid']!=0):
                    df.ix[i,'rid']=df.ix[j,'rid']
                    impute_flag=1
                    print("RID is 0 in row #{} ".format(i))
                    print df.ix[i]
                    print("RID in row #{} can be imputed by row #{} ".format(i,j))
                    print df.ix[j]
                    print("\n")
                    break
#             if(impute_flag==0):
#                 while(not (new_rid in list(df['rid']))):
#                     new_rid+=1
#                 df.ix[i,'rid']=new_rid
#                 new_rid+=1
#                 print("RID in row #{} is re-assigned as {}".format(i, new_rid))
#                 impute_flag=1
            if(impute_flag==0):
                print("No way to impute RID in row #{} ".format(i))
                print df.ix[i]
                print("\n")
                df.drop(i, inplace=True)
    #     elif(df.ix[i,'play_len'] ==0):
    #         print("Play_len is NA in row #{} ".format(i))
    #         print df.ix[i]
    #         print("\n")
    #         zero_list.append(i)
        elif(df.ix[i,'song_len'] ==0):
            impute_flag1=0
            for index, j in df.iterrows():
                    if(df.ix[j,'rid']==df.ix[i,'rid'] and j!=i and df.ix[j,'song_len']!=0):
                        df.ix[i,'song_len']=df.ix[j,'song_len']
                        impute_flag1=1
#                         print("Song_len is 0 in row #{} ".format(i))
#                         print df.ix[i]
                        print("Song_len in row #{} can be imputed by row #{} ".format(i,j))
                        print df.ix[j]
                        print("\n")
                        break
            if(impute_flag1==0):
                print("No way to impute song_len in row #{}".format(i))
                print df.ix[i]
                print("\n")
                df.drop(i, inplace=True)
    
# zero_list=song_len_zero_list + uid_zero_list + rid_zero_list
# for j in range(len(zero_list)):
#         df.drop(zero_list[j], inplace=True)
# zero_list=zero_list+null_list

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [11]:
#Hard version: impute "0" uid and rid rows as much as possible
import numpy as np

uid_zero_list=[]
rid_zero_list=[]
song_len_zero_list=[]
new_rid=1
new_uid=1
for i in range(row_num):
    if(not(i in null_list)):
        if(df.ix[i,'uid']==0): #re-assign uid=0 rows
#            impute_flag0=0
            while(not (new_rid in list(df['uid']))):
                new_uid+=1
            df.ix[i,'uid']=new_uid
            new_uid+=1
            print("UID in row #{} is re-assigned as {}".format(i, new_uid))
#             impute_flag0=1
#             if(impute_flag0==0):
#                 print("No way to impute UID in row #{} ".format(i))
#                 print df.ix[i]
#                 print("\n")
#                 uid_zero_list.append(i)
        elif(df.ix[i,'rid']==0): #impute rid=0 rows
            impute_flag=0
            for j in range(row_num):
                if(not(j in null_list)):
                    if(df.ix[j,'name']==df.ix[i,'name'] and j!=i and df.ix[j,'artist'] == df.ix[i,'artist'] and df.ix[j,'rid']!=0):
                        df.ix[i,'rid']=df.ix[j,'rid']
                        impute_flag=1
                        print("RID is 0 in row #{} ".format(i))
                        print df.ix[i]
                        print("RID in row #{} can be imputed by row #{} ".format(i,j))
                        print df.ix[j]
                        print("\n")
                        break
            if(impute_flag==0):
                while(not (new_rid in list(df['rid']))):
                    new_rid+=1
                df.ix[i,'rid']=new_rid
                new_rid+=1
                print("RID in row #{} is re-assigned as {}".format(i, new_rid))
                impute_flag=1
            if(impute_flag==0):
                print("No way to impute RID in row #{} ".format(i))
                print df.ix[i]
                print("\n")
                rid_zero_list.append(i)
    #     elif(df.ix[i,'play_len'] ==0):
    #         print("Play_len is NA in row #{} ".format(i))
    #         print df.ix[i]
    #         print("\n")
    #         zero_list.append(i)
        elif(df.ix[i,'song_len'] ==0):
            impute_flag1=0
            for j in range(row_num):
                if(not(j in null_list)):
                    if(df.ix[j,'rid']==df.ix[i,'rid'] and j!=i and df.ix[j,'song_len']!=0):
                        df.ix[i,'song_len']=df.ix[j,'song_len']
                        impute_flag1=1
                        print("Song_len is 0 in row #{} ".format(i))
                        print df.ix[i]
                        print("Song_len in row #{} can be imputed by row #{} ".format(i,j))
                        print df.ix[j]
                        print("\n")
            if(impute_flag1==0):
    #             print("Song_len is 0 in row #{} ".format(i))
    #             print df.ix[i]
    #             print("\n")
                song_len_zero_list.append(i)
    
zero_list=song_len_zero_list + uid_zero_list + rid_zero_list
for j in range(len(zero_list)):
        df.drop(zero_list[j], inplace=True)
zero_list=zero_list+null_list

RID in row #16 is re-assigned as 54395
RID in row #41 is re-assigned as 54740
RID in row #81 is re-assigned as 54917
RID in row #105 is re-assigned as 55492
RID in row #129 is re-assigned as 55524
Song_len is 0 in row #153 
uid         1.54423e+08
os                  ip 
rid         1.24566e+06
type                 0 
name                泡沫 
artist       G.E.M.邓紫棋 
play_len              0
song_len            258
TBA                   0
Name: 153, dtype: object
Song_len in row #153 can be imputed by row #232 
uid         1.54421e+08
os                  ip 
rid         1.24566e+06
type                 0 
name                泡沫 
artist       G.E.M.邓紫棋 
play_len            258
song_len            258
TBA                   0
Name: 232, dtype: object


Song_len is 0 in row #153 
uid         1.54423e+08
os                  ip 
rid         1.24566e+06
type                 0 
name                泡沫 
artist       G.E.M.邓紫棋 
play_len              0
song_len            258
TBA                   0


KeyboardInterrupt: 

In [70]:
song_len_zero_list, uid_zero_list, rid_zero_list, zero_list


([5,
  15,
  27,
  33,
  55,
  65,
  136,
  177,
  181,
  234,
  238,
  243,
  273,
  299,
  348,
  367,
  374,
  395,
  400,
  407,
  409,
  411,
  417,
  421,
  509,
  545,
  552,
  615,
  625,
  661,
  665,
  673,
  691,
  701,
  727,
  738,
  845,
  878,
  925,
  932,
  951,
  959,
  964,
  985,
  987,
  996,
  1010,
  1012,
  1029,
  1043,
  1103,
  1113,
  1118,
  1121,
  1123,
  1126,
  1134,
  1138,
  1143,
  1145,
  1148,
  1153,
  1158,
  1161,
  1200,
  1238,
  1339,
  1352,
  1376,
  1417,
  1443,
  1496,
  1513,
  1525,
  1560,
  1570,
  1586,
  1637,
  1701,
  1739,
  1747,
  1798,
  1824,
  1831,
  1956,
  1960,
  1987,
  1999,
  2003,
  2114,
  2128,
  2185,
  2200,
  2217,
  2233,
  2309,
  2311,
  2321,
  2337,
  2343,
  2393,
  2408,
  2410,
  2418,
  2475,
  2516,
  2527,
  2556,
  2566,
  2609,
  2619,
  2641,
  2679,
  2691,
  2709,
  2718,
  2764,
  2765,
  2766,
  2785,
  2808,
  2826,
  2847,
  2852,
  2858,
  2862,
  2898,
  2939,
  2952,
  2976,
  3018,
  302

In [71]:
df.shape

(4544, 9)

In [72]:
df['uid'].astype(int)
#df['play_len'].astype(int)
#df['song_len'].astype(int)
df.shape
df.head()

,uid,os,rid,type,name,artist,play_len,song_len,TBA
0,154422682.0,ar,20870993.0,1,用情,狮子合唱团,22013.0,332.0,0.0
1,154421907.0,ip,6560858.0,0,表情不要悲伤,伯贤&D.O.&张艺兴&朴灿烈,96.0,161.0,0.0
2,154422630.0,ar,3385963.0,1,"Baby, Don't Cry(人鱼的眼泪)",EXO,235868.0,235.0,0.0
3,154410267.0,ar,6777172.0,0,3D-环绕音律1(3D Mix),McTaiM,164.0,237.0,0.0
4,154407793.0,ar,19472465.0,0,刚好遇见你,曲肖冰,24.0,201.0,0.0


In [73]:
df['play_times'] = np.zeros(df.shape[0])
for i in range(row_num):
    if(not(i in zero_list)):
        df.ix[i,'play_times']=df.ix[i,'play_len']/df.ix[i,'song_len']
df.head()

,uid,os,rid,type,name,artist,play_len,song_len,TBA,play_times
0,154422682.0,ar,20870993.0,1,用情,狮子合唱团,22013.0,332.0,0.0,66.304217
1,154421907.0,ip,6560858.0,0,表情不要悲伤,伯贤&D.O.&张艺兴&朴灿烈,96.0,161.0,0.0,0.596273
2,154422630.0,ar,3385963.0,1,"Baby, Don't Cry(人鱼的眼泪)",EXO,235868.0,235.0,0.0,1003.693617
3,154410267.0,ar,6777172.0,0,3D-环绕音律1(3D Mix),McTaiM,164.0,237.0,0.0,0.691983
4,154407793.0,ar,19472465.0,0,刚好遇见你,曲肖冰,24.0,201.0,0.0,0.119403


In [74]:
df.ix[0,'play_times']
df.describe()
num_mult=0
num_mult_5=0
num_mult_10=0
num_mult_20=0
num_mult_100=0
for i in range(df.shape[0]):
    if(not(i in zero_list)):
        if(df.ix[i,'play_times'] > 1):
            num_mult=num_mult+1
        if(df.ix[i,'play_times'] > 5):
            num_mult_5=num_mult_5+1
        if(df.ix[i,'play_times'] > 10):
            num_mult_10=num_mult_10+1
        if(df.ix[i,'play_times'] > 20):
            num_mult_20=num_mult_20+1
        if(df.ix[i,'play_times'] > 100):
            num_mult_100=num_mult_100+1
            
for i in range(df.shape[0]):
    if(not(i in zero_list)):
        if(df.ix[i,'play_times'] >= 5):
            df.ix[i,'play_times']=5 #Cap to 5 times 
            df.ix[i,'play_len']= df.ix[i,'song_len']*5
            
num_mult, num_mult_5, num_mult_10, num_mult_20, num_mult_100

(392, 250, 246, 243, 226)

In [75]:
#uid_list = np.zeros(df.shape[0])
#rid_list = np.zeros(df.shape[0])
repeate_list=[]
for i in range(df.shape[0]):
#    if(not(i in zero_list) and not(i in repeate_list)):
    if(not(i in zero_list)):
        for j in range(i):
            if(not(j in zero_list)):
                if((df.ix[i,'uid']==df.ix[j,'uid']) and (df.ix[i,'rid']==df.ix[j,'rid'])):
                    df.ix[j,'play_times']+= df.ix[i,'play_times']#drop row i and add play_times to row j
                    if(not(i in repeate_list)):
                        repeate_list.append(i)
                    if(df.ix[i,'song_len']==df.ix[j,'song_len']):
                        df.ix[j,'play_len']+= df.ix[i,'play_len']
                    else:
                        print("Row #{} and row #{} have the same uid{} and rid{}, \
                              but different song length".format(i,j, df.ix[i,'uid'], df.ix[i,'rid']))

for i in range(len(repeate_list)):
    df.drop(repeate_list[i], axis=0, inplace=True)
                               

Row #422 and row #233 have the same uid154422852.0 and rid20862199.0,                               but different song length
Row #430 and row #245 have the same uid154422850.0 and rid20862199.0,                               but different song length
Row #458 and row #257 have the same uid154422858.0 and rid20862199.0,                               but different song length
Row #465 and row #254 have the same uid154422860.0 and rid20862199.0,                               but different song length
Row #624 and row #470 have the same uid154422876.0 and rid20862199.0,                               but different song length
Row #679 and row #315 have the same uid154422750.0 and rid466227.0,                               but different song length
Row #947 and row #708 have the same uid154422901.0 and rid20862199.0,                               but different song length
Row #1003 and row #772 have the same uid154422904.0 and rid20862199.0,                               but different song 

In [76]:
len(repeate_list)

431

In [13]:

for _, row in df.iterrows():
    # subtract 1 from id's due to match 0 indexing
    utility_mat[row.user-1, row.movie-1] = row.rating

,uid,os,rid,type,name,artist,play_len,song_len,TBA
0,154422682.0,ar,20870993,1,用情,狮子合唱团,22013,332,0.0
1,154421907.0,ip,6560858,0,表情不要悲伤,伯贤&D.O.&张艺兴&朴灿烈,96,161,0.0
2,154422630.0,ar,3385963,1,"Baby, Don't Cry(人鱼的眼泪)",EXO,235868,235,0.0
3,154410267.0,ar,6777172,0,3D-环绕音律1(3D Mix),McTaiM,164,237,0.0
4,154407793.0,ar,19472465,0,刚好遇见你,曲肖冰,24,201,0.0


In [77]:
utility_mat = pd.pivot_table(data=df, 
                            values='play_times', 
                            index='uid', 
                            columns='rid', 
                            fill_value=0)


In [119]:
utility_mat.ix[168955.0, ].nonzero()


(array([1099, 2626]),)

In [82]:
from sklearn.metrics.pairwise import cosine_similarity
# Item-Item Similarity Matrix
item_sim_mat = cosine_similarity(utility_mat.T)
item_sim_mat.shape

(2727, 2727)

In [83]:
least_to_most_sim_indexes = np.argsort(item_sim_mat, axis=1)

# Neighborhoods
neighborhood_size = 75
neighborhoods = least_to_most_sim_indexes[:, -neighborhood_size:]

In [157]:
len(neighborhoods[5])

75

In [102]:
uid = 168955.0

n_users = utility_mat.shape[0]
n_items = utility_mat.shape[1]

items_played_by_this_user = utility_mat.ix[100, 1].nonzero()[1]
items_played_by_this_user

KeyError: 100.0

In [151]:
utility_mat.loc[168955].values

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [158]:
neighborhoods.shape[1]

75

In [159]:
# Just initializing so we have somewhere to put rating preds
out = np.zeros(n_items)
for item_to_play in xrange(n_items):
#     relevant_items = np.intersect1d(neighborhoods[item_to_play],
#                                     items_played_by_this_user,
#                                     assume_unique=True)  # assume_unique speeds up intersection op
    if(utility_mat.loc[uid].values[item_to_play]==0):
        for neighbor_id in range(neighborhoods.shape[1]):
             out[item_to_play] = out[item_to_play] + \
                utility_mat.loc[uid].values[neighborhoods[item_to_play, neighbor_id]] *\
                item_sim_mat[item_to_play, neighborhoods[item_to_play, neighbor_id]]
        out[item_to_play] = out[item_to_play] / item_sim_mat[item_to_play, ].sum()
                
pred_play_times = np.nan_to_num(out)
print pred_play_times
#print("Execution time: %f seconds" % (time()-start_time))

[ 0.  0.  0. ...,  0.  0.  0.]


In [153]:
utility_mat.loc[uid].values[5]

0.0